# Основы парсинга

Булыгин Олег:  
* [LinkedIn](linkedin.com/in/obulygin)  
* [Мой канал в ТГ по Python](https://t.me/pythontalk_ru)

In [1]:
import pandas as pd

In [2]:
import requests

In [3]:
# метод get
res = requests.get('https://netology.ru/blog/')
# res
# res.status_code

In [ ]:
# браузер отрисовал бы страницу на основе данного текста
res.text

In [5]:
# получаем плохой статус
bad_request = requests.get('https://netology.ru/blog/some_page')
bad_request

<Response [404]>

In [ ]:
bad_request.text

In [7]:
# как разбирать всю эту разметку? Поможет BeautifulSoup.
from bs4 import BeautifulSoup

In [ ]:
# создаем объекта, через методы которого будем искать нужные теги и извлекать их содержимое
soup = BeautifulSoup(res.text)
soup

In [9]:
# функция find_all позволяет найти все указанные теги с нужными атрибутами (с вложениями), возвращает список
news = soup.find_all('div', class_='posts__item')
print(len(news))
print(news[0])

15
<div class="posts__item" id="post-38665">
<div class="posts__img">
<a href="https://netology.ru/blog/11-2022-edmarket-collaboration-online" target="_blank">
<img alt="8 онлайн-сервисов для совместной работы над учебными проектами" src="https://blog-prod-bucket.website.yandexcloud.net/uploads/2022/11/Netology-Cover-Collaboration.jpg"/>
</a>
<div class="tags">
<a class="tags__item" href="/blog/category/what-is" style="background:#eb236b"><span style="background:#eb236b">Разобраться</span></a>
</div>
</div>
<h2 class="posts__title"><a class="posts__link" href="https://netology.ru/blog/11-2022-edmarket-collaboration-online" target="_blank">8 онлайн-сервисов для совместной работы над учебными проектами</a></h2>
<div class="posts__txt">
</div>
<div class="posts__meta">
<div class="posts__date">25 ноября 2022</div>
</div>
</div>


In [10]:
for el in news:
    title = el.find('a', 'posts__link').text
    print(title)
    link = el.find('a', 'posts__link').get('href')
    print(link)
    date = el.find('div', 'posts__date').text
    print(date)
    category = el.find('span').text
    print(category)
    print()

8 онлайн-сервисов для совместной работы над учебными проектами
https://netology.ru/blog/11-2022-edmarket-collaboration-online
25 ноября 2022
Разобраться

Вместо Canva: 7 онлайн-сервисов для создания презентаций
https://netology.ru/blog/11-2022-presentation-services
24 ноября 2022
Продуктивность

Как понять, что выгорел, и справиться с этим
https://netology.ru/blog/11-2022-burnout-control
22 ноября 2022
Продуктивность

Решения для корпоративного обучения: какое выбрать и от чего зависит стоимость
https://netology.ru/blog/11-2022-resheniya-dlya-korporativnogo-obucheniya
21 ноября 2022
Разобраться

«Сожалений о выбранном пути не было ни разу»: выпускница EdMarket — о смене курса в профессии и погружении в онлайн
https://netology.ru/blog/11-2022-edmarket-life-on-the-hook
18 ноября 2022
Личный опыт

10 книг о самообразовании: как учиться новому быстро и эффективно
https://netology.ru/blog/11-2022-books-about-self-education
17 ноября 2022
Книжная полка

Как справляться с негативными эмоциями

Напишем функцию на основе всех предыдущих действий, которая будет возвращать датафрейм с датой, заголовком, категорией и ссылкой на полный текст поста


In [12]:
def get_netology_blog_posts():
    # define an url
    url = 'https://netology.ru/blog/'
    # send a request
    req = requests.get(url).text
    # feed the result into soup
    soup = BeautifulSoup(req)

    # find apropriate tag and class in HTML
    news = soup.find_all('div', class_='posts__item')

    # declare s df
    netology_blog = pd.DataFrame()

    # record the df
    for el in news:
        title = el.find('a', 'posts__link').text
        link = el.find('a', 'posts__link').get('href')
        date = el.find('div', 'posts__date').text
        category = el.find('a', 'tags__item').text
        # form a row
        row = {'date': date, 'title': title, 'link': link, 'category':category}
        # append row to df
        netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 

        # reset index to avoid zero indexes (and drop index column)
    return netology_blog.reset_index(drop=True)

get_netology_blog_posts()

,date,title,link,category
0,25 ноября 2022,8 онлайн-сервисов для совместной работы над уч...,https://netology.ru/blog/11-2022-edmarket-coll...,Разобраться
1,24 ноября 2022,Вместо Canva: 7 онлайн-сервисов для создания п...,https://netology.ru/blog/11-2022-presentation-...,Продуктивность
2,22 ноября 2022,"Как понять, что выгорел, и справиться с этим",https://netology.ru/blog/11-2022-burnout-control,Продуктивность
3,21 ноября 2022,Решения для корпоративного обучения: какое выб...,https://netology.ru/blog/11-2022-resheniya-dly...,Разобраться
4,18 ноября 2022,«Сожалений о выбранном пути не было ни разу»: ...,https://netology.ru/blog/11-2022-edmarket-life...,Личный опыт
5,17 ноября 2022,10 книг о самообразовании: как учиться новому ...,https://netology.ru/blog/11-2022-books-about-s...,Книжная полка
6,16 ноября 2022,Как справляться с негативными эмоциями и управ...,https://netology.ru/blog/11-2022-how-to-manage...,Продуктивность
7,15 ноября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/11-2022-data-science-...,Книжная полка
8,15 ноября 2022,Ручное тестирование лендинга: что нужно знать ...,https://netology.ru/blog/11-2022-manual-testin...,Продуктивность
9,14 ноября 2022,Негативные эмоции в онлайн-обучении: как помог...,https://netology.ru/blog/11-2022-edmarket-chal...,Разобраться


А теперь поработаем поиском по сайту

In [13]:
# cформируем поисковый запрос, обратите внимание на его формат
URL = 'https://netology.ru/blog/?s=python'

In [14]:
req = requests.get(URL)
req.text

'<!DOCTYPE html>\n<html lang="ru-RU" prefix="og: https://ogp.me/ns#">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="https://gmpg.org/xfn/11">\n<title>python | Медиа Нетологии</title>\n\n<meta name="robots" content="max-image-preview:large" />\n<link rel="canonical" href="https://netology.ru/blog?s=python" />\n<link rel="next" href="https://netology.ru/blog/page/2?s=python" />\n\n<link rel="preload" href="https://netology.ru/blog/wp-content/plugins/rate-my-post/public/css/fonts/ratemypost.ttf" type="font/ttf" as="font" crossorigin="anonymous">\n\n<script data-cfasync="false" data-pagespeed-no-defer type="text/javascript">//<![CDATA[\n\tvar gtm4wp_datalayer_name = "dataLayer";\n\tvar dataLayer = dataLayer || [];\n//]]>\n</script>\n<link rel=\'dns-prefetch\' href=\'//netology.ru\' />\n<link rel=\'dns-prefetch\' href=\'//fonts.googleapis.com\' />\n<link rel=\'dns-prefetch\' href=\'//s.w.org\' />\n<link rel=\

В функцию get можно передавать параметры и заголовки запроса в виде словаря

In [15]:
# в request можно передать параметры запроса и заголовки (headers) в виде словарей. 
URL = 'https://netology.ru/blog/'
params = {
    's': 'python'
}
# headers not obligatory parameter
headers = {
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Mobile Safari/537.36'
}

# send a request
req = requests.get(URL, params=params, headers=headers)

req.text

'<!DOCTYPE html>\n<html lang="ru-RU" prefix="og: https://ogp.me/ns#">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<link rel="profile" href="https://gmpg.org/xfn/11">\n<title>python | Медиа Нетологии</title>\n\n<meta name="robots" content="max-image-preview:large" />\n<link rel="canonical" href="https://netology.ru/blog?s=python" />\n<link rel="next" href="https://netology.ru/blog/page/2?s=python" />\n\n<link rel="preload" href="https://netology.ru/blog/wp-content/plugins/rate-my-post/public/css/fonts/ratemypost.ttf" type="font/ttf" as="font" crossorigin="anonymous">\n\n<script data-cfasync="false" data-pagespeed-no-defer type="text/javascript">//<![CDATA[\n\tvar gtm4wp_datalayer_name = "dataLayer";\n\tvar dataLayer = dataLayer || [];\n//]]>\n</script>\n<link rel=\'dns-prefetch\' href=\'//netology.ru\' />\n<link rel=\'dns-prefetch\' href=\'//fonts.googleapis.com\' />\n<link rel=\'dns-prefetch\' href=\'//s.w.org\' />\n<link rel=\

In [16]:
soup = BeautifulSoup(req.text)
# main part. find the right tag and class
articles = soup.find_all('article', class_='status-publish')
len(articles)

6

Напишем функция для извлечения постов по несколькоим запросам со страницы

In [17]:
import time  # будем ставить задержку между запросами

def get_netology_posts(query):
    # declare a df
    netology_blog = pd.DataFrame()
    # define an URL
    URL = 'https://netology.ru/blog/'
    # itereate thr. list of requests
    for q in query:
        params = {
            # s - search
            's': q
        }
        # send a request
        req = requests.get(URL, params=params)
        # pause to emulate human-like actions
        time.sleep(0.3)
        # feed the soup
        soup = BeautifulSoup(req.text)
        # news wrapped into 'article', class_='status-publish'
        articles = soup.find_all('article', class_='status-publish')
        # iterations
        for article in articles:

            title = article.find('h2', class_='entry-title').text
            # look within h2 for href
            link = article.find('h2', class_='entry-title').find('a').get('href')
            date = article.find('span', class_='posted-on').text.strip()
            category = article.find('div', class_='entry-cats').text
            row = {'date': date, 'title': title, 'link': link, 'category':category}
            netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
    return netology_blog.reset_index(drop=True)

get_netology_posts(['python', 'data science'])

AttributeError: ignored

one block has no category

In [18]:
def get_netology_posts(query):
    netology_blog = pd.DataFrame()
    for q in query:
        URL = 'https://netology.ru/blog/'
        params = {
            's': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='status-publish')
        for article in articles:
            title = article.find('h2', class_='entry-title').text
            link = article.find('h2', class_='entry-title').find('a').get('href')
            date = article.find('span', class_='posted-on').text.strip()
            # if find
            if article.find('div', class_='entry-cats'):
                category = article.find('div', class_='entry-cats').text
            else:
                # haven't found
                category = 'Новость'
            row = {'date': date, 'title': title, 'link': link, 'category':category}
            netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
    return netology_blog.reset_index(drop=True)
# 
res = get_netology_posts(['python', 'data science'])
res

,date,title,link,category
0,12 октября 2021,Python занял первое место в рейтинге языков пр...,https://netology.ru/blog/news/12-10-2021-python,Новость
1,31 июля 2019,С чего начать изучение Python: 3 первых шага,https://netology.ru/blog/07-2019-python-3-perv...,Современные профессии
2,15 марта 2019,С чего начать изучение Python: книги для начин...,https://netology.ru/blog/03-2019-python-knigi-...,Книжная полка
3,16 января 2019,"Как перейти в Machine learning, если знаешь Py...",https://netology.ru/blog/2019-01-machine-learn...,Карьера
4,15 января 2019,Создание модели распознавания лиц с использова...,https://netology.ru/blog/2019-01-modeli-raspoz...,Руководство
5,30 декабря 2016,Как парсить данные с HTML-страниц на Python,https://netology.ru/blog/parsing-python,Руководство
6,15 ноября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/11-2022-data-science-...,Книжная полка
7,12 октября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/10-2022-data-science-...,Книжная полка
8,13 сентября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/09-2022-data-science-...,Книжная полка
9,17 августа 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/08-2022-data-science-...,Книжная полка


А если хотим добавить в датафрейм полные тексты всех постов?

In [ ]:
def add_full_text(posts_df):
    i = 0
    for el in posts_df['link']:
        print(el)
        # send a request
        req = requests.get(el).text
        # feed the soup
        soup = BeautifulSoup(req)
        # pause
        time.sleep(0.3)
        # fins full text
        full_text = soup.find('div', class_='entry-content').text.strip()
        # put search result into new column 'text'
        posts_df.loc[i, 'text'] = full_text
        i += 1
    return posts_df

add_full_text(res)

https://netology.ru/blog/news/12-10-2021-python
https://netology.ru/blog/07-2019-python-3-pervykh-shaga
https://netology.ru/blog/03-2019-python-knigi-novichkam
https://netology.ru/blog/2019-01-machine-learning-python
https://netology.ru/blog/2019-01-modeli-raspoznavaniya-lic
https://netology.ru/blog/parsing-python
https://netology.ru/blog/11-2022-data-science-digest
https://netology.ru/blog/10-2022-data-science-digest
https://netology.ru/blog/09-2022-data-science-digest
https://netology.ru/blog/08-2022-data-science-digest
https://netology.ru/blog/07-2022-data-science-digest
https://netology.ru/blog/06-2022-data-science-digest


,date,title,link,category,text
0,12 октября 2021,Python занял первое место в рейтинге языков пр...,https://netology.ru/blog/news/12-10-2021-python,Новость,Впервые за 20 лет Python стал самым популярным...
1,31 июля 2019,С чего начать изучение Python: 3 первых шага,https://netology.ru/blog/07-2019-python-3-perv...,Современные профессии,"В прошлой статье мы рассказали, почему стоит и..."
2,15 марта 2019,С чего начать изучение Python: книги для начин...,https://netology.ru/blog/03-2019-python-knigi-...,Книжная полка,"Нетология рассказала, с какой книги начать изу..."
3,16 января 2019,"Как перейти в Machine learning, если знаешь Py...",https://netology.ru/blog/2019-01-machine-learn...,Карьера,Директор разработки и data scientist ДомКлик.р...
4,15 января 2019,Создание модели распознавания лиц с использова...,https://netology.ru/blog/2019-01-modeli-raspoz...,Руководство,"Адаптировали статью Файзана Шайха о том, как с..."
5,30 декабря 2016,Как парсить данные с HTML-страниц на Python,https://netology.ru/blog/parsing-python,Руководство,Руководитель проектов Дмитрий Мирошниченко рас...
6,15 ноября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/11-2022-data-science-...,Книжная полка,"В новом дайджесте рассказываем, как ИИ взламыв..."
7,12 октября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/10-2022-data-science-...,Книжная полка,"Делаем покемонов из кого угодно, смотрим «Ла-Л..."
8,13 сентября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/09-2022-data-science-...,Книжная полка,"Проверяем себя на знание мемов, создаём фон в ..."
9,17 августа 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/08-2022-data-science-...,Книжная полка,"В новом дайджесте рассказываем, как ИИ помогае..."


Напишем скрипт, который будет собирать новости с сайта Коммерсанта

In [ ]:
URL = 'https://www.kommersant.ru/search/results'
params = {
    'search_query': 'python'
}

In [ ]:
res = requests.get(URL, params)

In [ ]:
res.text

'\r\n<!DOCTYPE html>\r\n\r\n<html class="no-js"  lang="ru">\r\n<head>\r\n    <title>Коммерсантъ: последние новости России и мира</title>\r\n    <meta http-equiv="X-UA-Compatible" content="IE=edge, chrome=1" />\r\n    <meta charset="utf-8" />\r\n\r\n    <meta name="format-detection" content="telephone=no" />\r\n    <meta name="title" content="Коммерсантъ: последние новости России и мира" />\r\n    <meta name="description" content="Актуальные новости, объективный анализ и эксклюзивные комментарии о важнейших событиях и трендах" />\r\n    <meta name="keywords" content="Новости,Политика,Экономика,Бизнес,Финансы,Дело,Биржа,Рынок,Акции,Прогнозы,Критика,Интервью,Рейтинги,Документы,Деньги,Власть,Автопилот,Тематические страницы,Первые лица,Деловые новости,Мировая практика,Культура,Спорт,Weekend,Астрологический прогноз,Погода мира,Курсы валют ЦБ РФ" />\r\n\r\n    <meta name="yandex-verification" content="50df68945a519dbd" />\r\n\r\n    \r\n<meta name="viewport" content="width=device-width, initi

In [ ]:
soup = BeautifulSoup(res.text)
soup

<!DOCTYPE html>
<html class="no-js" lang="ru">
<head>
<title>Коммерсантъ: последние новости России и мира</title>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="Коммерсантъ: последние новости России и мира" name="title"/>
<meta content="Актуальные новости, объективный анализ и эксклюзивные комментарии о важнейших событиях и трендах" name="description"/>
<meta content="Новости,Политика,Экономика,Бизнес,Финансы,Дело,Биржа,Рынок,Акции,Прогнозы,Критика,Интервью,Рейтинги,Документы,Деньги,Власть,Автопилот,Тематические страницы,Первые лица,Деловые новости,Мировая практика,Культура,Спорт,Weekend,Астрологический прогноз,Погода мира,Курсы валют ЦБ РФ" name="keywords"/>
<meta content="50df68945a519dbd" name="yandex-verification"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="324580610921010" property="fb:app_id"/>
<meta content="website" propert

In [ ]:
# собираем все теги со ссылками на полные тексты новостей
refs = soup.find_all('a', class_='uho__link')
print(len(refs))
print(refs)

24
[<a class="uho__link uho__link--overlay" href="/doc/5607909?query=python" target="_blank">GeekBrains: россияне стали чаще учить английский и интересоваться IT</a>, <a class="uho__link" href="/doc/5607909?query=python" target="_blank"> ... % чаще. Увеличилась популярность <mark>Python</mark> (на 15%), веб ... </a>, <a class="uho__link uho__link--overlay" href="/doc/5581270?query=python" target="_blank">Скиллы c оглядкой на локацию</a>, <a class="uho__link" href="/doc/5581270?query=python" target="_blank">Кадры</a>, <a class="uho__link" href="/doc/5581270?query=python" target="_blank"> ...  меньшим спросом), DevOps, <mark>Python</mark>, разработчиков ПО, тестировщиков ... </a>, <a class="uho__link uho__link--overlay" href="/doc/5547697?query=python" target="_blank">Айтишники в тренде</a>, <a class="uho__link" href="/doc/5547697?query=python" target="_blank"> ...  тренде — разработка на <mark>Python</mark> (+54%), CSS (+48 ... </a>, <a class="uho__link uho__link--overlay" href="/doc/55

In [ ]:
# добираемся до ссылок
all_links = []

for ref in refs:
    # get links     .get()
    all_links.append(ref.get('href'))

print(len(all_links))
print(all_links)

24
['/doc/5607909?query=python', '/doc/5607909?query=python', '/doc/5581270?query=python', '/doc/5581270?query=python', '/doc/5581270?query=python', '/doc/5547697?query=python', '/doc/5547697?query=python', '/doc/5526837?query=python', '/doc/5526837?query=python', '/doc/5501418?query=python', '/doc/5501418?query=python', '/doc/5501418?query=python', '/doc/5478996?query=python', '/doc/5478996?query=python', '/doc/5392083?query=python', '/doc/5392083?query=python', '/doc/5379833?query=python', '/doc/5379833?query=python', '/doc/5379833?query=python', '/doc/5338616?query=python', '/doc/5338616?query=python', '/doc/5338616?query=python', '/doc/5328418?query=python', '/doc/5328418?query=python']


In [ ]:
# исключаем дубли
all_links = set(all_links)
print(len(all_links))
print(all_links)

10
{'/doc/5526837?query=python', '/doc/5581270?query=python', '/doc/5501418?query=python', '/doc/5379833?query=python', '/doc/5547697?query=python', '/doc/5478996?query=python', '/doc/5392083?query=python', '/doc/5607909?query=python', '/doc/5338616?query=python', '/doc/5328418?query=python'}


In [ ]:
# формируем полноценные ссылки
# add prefix for links
all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))
print(all_full_links)

['https://www.kommersant.ru/doc/5526837?query=python', 'https://www.kommersant.ru/doc/5581270?query=python', 'https://www.kommersant.ru/doc/5501418?query=python', 'https://www.kommersant.ru/doc/5379833?query=python', 'https://www.kommersant.ru/doc/5547697?query=python', 'https://www.kommersant.ru/doc/5478996?query=python', 'https://www.kommersant.ru/doc/5392083?query=python', 'https://www.kommersant.ru/doc/5607909?query=python', 'https://www.kommersant.ru/doc/5338616?query=python', 'https://www.kommersant.ru/doc/5328418?query=python']


In [ ]:
# объединим все в одну функцию
def get_all_links(query):
    url = 'https://www.kommersant.ru/search/results'
    params = {
        'search_query': query,
    }
    res = requests.get(url, params)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text)
    refs = soup.find_all('a', class_='uho__link')

    all_links = []
    for ref in refs:
        all_links.append(ref.get('href'))
    # remove duplicates
    all_links = set(all_links)

    all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))

    return all_full_links

all_links = get_all_links('python')
print(all_links)

['https://www.kommersant.ru/doc/5526837?query=python', 'https://www.kommersant.ru/doc/5581270?query=python', 'https://www.kommersant.ru/doc/5501418?query=python', 'https://www.kommersant.ru/doc/5379833?query=python', 'https://www.kommersant.ru/doc/5547697?query=python', 'https://www.kommersant.ru/doc/5478996?query=python', 'https://www.kommersant.ru/doc/5392083?query=python', 'https://www.kommersant.ru/doc/5607909?query=python', 'https://www.kommersant.ru/doc/5338616?query=python', 'https://www.kommersant.ru/doc/5328418?query=python']


In [ ]:
# но мы же собрали только одну страницу? Хотим ВСЕ новости
def get_all_links(query, pages):
    url = 'https://www.kommersant.ru/search/results'
    links_list = []
    # search dict
    # {'search_query': 'python', 'page': 1}
    # {'search_query': 'python', 'page': 2}
    # {'search_query': 'python', 'page': 3}
    params = {
        'search_query': query
    }
    for i in range(1, pages+1):
        # add key, value   
        # 'page': 1
        params['page'] = i
        # print (params)
        # send request with params
        res = requests.get(url, params)
        # feed the soup
        soup = BeautifulSoup(res.text)
        # print(res.text)
        time.sleep(0.33)
        # find refs list
        refs = soup.find_all('a', class_='uho__link')
        all_links = []

        # iterate refs
        for ref in refs:
            # yield a row
            all_links.append(ref.get('href'))
        # remove duplicates
        all_links = set(all_links)

        # add a prefix
        all_full_links = list(map(lambda x: 'https://www.kommersant.ru' + x, all_links))

        # extend links_list
        links_list += all_full_links
    return links_list

all_links = get_all_links('python', 3)
print(len(all_links))
print(all_links)

25
['https://www.kommersant.ru/doc/5526837?query=python', 'https://www.kommersant.ru/doc/5581270?query=python', 'https://www.kommersant.ru/doc/5501418?query=python', 'https://www.kommersant.ru/doc/5379833?query=python', 'https://www.kommersant.ru/doc/5547697?query=python', 'https://www.kommersant.ru/doc/5478996?query=python', 'https://www.kommersant.ru/doc/5392083?query=python', 'https://www.kommersant.ru/doc/5607909?query=python', 'https://www.kommersant.ru/doc/5338616?query=python', 'https://www.kommersant.ru/doc/5328418?query=python', 'https://www.kommersant.ru/doc/5179940?query=python', 'https://www.kommersant.ru/doc/5295067?query=python', 'https://www.kommersant.ru/doc/5171805?query=python', 'https://www.kommersant.ru/doc/5271237?query=python', 'https://www.kommersant.ru/doc/5238663?query=python', 'https://www.kommersant.ru/doc/5258915?query=python', 'https://www.kommersant.ru/doc/5292974?query=python', 'https://www.kommersant.ru/doc/5293079?query=python', 'https://www.kommersant.

In [ ]:
# now we have list of links of search request
# lets iterate
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.33)
    
    date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
    print(date)
    title = soup.find('h1', class_='doc_header__name').text.strip()
    print(title)
    text = soup.find('div', class_='doc__body').text.strip()
    print(text)

2022-08-26 10:32:54+03:00
«Яндекс» опубликовал инструмент для ускорения разработки мобильных приложений DivKit
Российская компания «Яндекс» выложила в открытый доступ фреймворк DivKit, позволяющий быстрее обновлять интерфейсы и ускорять разработку мобильных приложений. Исходный код инструмента был опубликован на GitHub.
При управлении интерфейсом приложения с сервера DivKit позволяет менять интерфейс на ходу, без обновления приложений. После описания интерфейса на сервере в специальном формате фреймворк превращает описание в элементы приложения одновременно для нескольких платформ: Android, iOS и веб-версий. 
С помощью инструмента можно, например, добавить счетчик лайков или галерею. В DivKit включены библиотеки для разработки серверной части на Kotlin, TypeScript и Python. «Яндекс» разрабатывал инструмент пять лет, в настоящее время он задействован в мобильном приложении «Яндекса», «Яндекс. Маркете», голосовом помощнике «Алиса» и других сервисах.
Евгений Федуненко
2022-09-29 00:20:00+

In [ ]:
# обернем в функцию 
def get_kom_news(links):
    # declare df
    kom_news = pd.DataFrame()
    # iterate all_links
    for link in all_links:
        # feed the request result
        soup = BeautifulSoup(requests.get(link).text)
        #pause
        time.sleep(0.33)
        
        date = pd.to_datetime(soup.find('time', class_='doc_header__publish_time').get('datetime'))
        title = soup.find('h1', class_='doc_header__name').text.strip()
        text = soup.find('div', class_='doc__body').text.strip().replace('\n', '')
        # form a row 
        row = {'date': date, 'title': title, 'text': text}
        # add the row to df
        kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
        # drop index (to avoid zero idxs) and index column
    return kom_news.reset_index(drop=True)

In [ ]:
kom_news = get_kom_news(all_links)
display(kom_news)

,date,title,text
0,2022-08-26 10:32:54+03:00,«Яндекс» опубликовал инструмент для ускорения ...,Российская компания «Яндекс» выложила в открыт...
1,2022-09-29 00:20:00+03:00,Скиллы c оглядкой на локацию,"В 2021 году в Минцифры подсчитали, что дефицит..."
2,2022-08-06 00:29:44+03:00,Стоило ли копии ломать,"Профильное сообщество, занимающееся развитием ..."
3,2022-05-31 00:01:00+03:00,Системный переход,Россию ожидает серьезный вызов на рынке IT-кад...
4,2022-09-06 13:43:35+03:00,Айтишники в тренде,Аналитики Price.ru и Авито Работы рассказали о...
5,2022-07-22 13:14:56+03:00,МТС увеличит количество ИТ-специалистов вдвое ...,До конца года компания планирует принять на ра...
6,2022-06-06 11:56:53+03:00,Две ИТ-компании включили в список системообраз...,В список системообразующих предприятий Удмурти...
7,2022-10-12 12:04:41+03:00,GeekBrains: россияне стали чаще учить английск...,Доля желающих освоить новую профессию выросла ...
8,2022-04-30 00:21:47+03:00,Запрыгнуть в последний Python,Россияне после начала военной операции на Укра...
9,2022-04-27 20:19:42+03:00,Кафедра виртуальных наук,В Пермском крае появится первый IT-вуз. О созд...


In [ ]:
def get_netology_posts(query):
    netology_blog = pd.DataFrame()
    for q in query:
        URL = 'https://netology.ru/blog/'
        params = {
            's': q
        }
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text)
        articles = soup.find_all('article', class_='status-publish')
        for article in articles:
            title = article.find('h2', class_='entry-title').text
            link = article.find('h2', class_='entry-title').find('a').get('href')
            date = article.find('span', class_='posted-on').text.strip()
            if article.find('div', class_='entry-cats'):
                category = article.find('div', class_='entry-cats').text
            else:
                category = 'Новость'
            row = {'date': date, 'title': title, 'link': link, 'category':category}
            netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
    return netology_blog.reset_index(drop=True)

res = get_netology_posts(['python', 'data science'])
res

,date,title,link,category
0,12 октября 2021,Python занял первое место в рейтинге языков пр...,https://netology.ru/blog/news/12-10-2021-python,Новость
1,31 июля 2019,С чего начать изучение Python: 3 первых шага,https://netology.ru/blog/07-2019-python-3-perv...,Современные профессии
2,15 марта 2019,С чего начать изучение Python: книги для начин...,https://netology.ru/blog/03-2019-python-knigi-...,Книжная полка
3,16 января 2019,"Как перейти в Machine learning, если знаешь Py...",https://netology.ru/blog/2019-01-machine-learn...,Карьера
4,15 января 2019,Создание модели распознавания лиц с использова...,https://netology.ru/blog/2019-01-modeli-raspoz...,Руководство
5,30 декабря 2016,Как парсить данные с HTML-страниц на Python,https://netology.ru/blog/parsing-python,Руководство
6,15 ноября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/11-2022-data-science-...,Книжная полка
7,12 октября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/10-2022-data-science-...,Книжная полка
8,13 сентября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/09-2022-data-science-...,Книжная полка
9,17 августа 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/08-2022-data-science-...,Книжная полка


In [ ]:
URL = 'https://netology.ru/blog/?infinity=scrolling'
params = {
    'page': 1,
    'query_args[s]': 'python'
}
req = requests.get(URL, params=params)


In [ ]:
req.json()['html']

'\n<article id="post-3192" class="post-3192 post type-post status-publish format-standard has-post-thumbnail hentry category-professions tag-dev">\n\t<a class="thumb-link" href="https://netology.ru/blog/07-2019-python-3-pervykh-shaga"><span role="img" aria-label="" class="featured-image has-background-cover has-animation" style="background-image: url(\'https://blog-prod-bucket.website.yandexcloud.net/uploads/2020/05/222joshua-coleman-unsplash-980x460.jpg\');"></span></a>\t<div class="hentry-inner">\n        <span class="posted-on">\n            <a href="https://netology.ru/blog/07-2019-python-3-pervykh-shaga" rel="bookmark">31 июля 2019</a>\n        </span>\n        <header class="entry-header primary-font">\n\t\t<div class="entry-cats"><a href="https://netology.ru/blog/category/professions" rel="category tag">Современные профессии</a></div><h2 class="entry-title"><a href="https://netology.ru/blog/07-2019-python-3-pervykh-shaga" rel="bookmark">С чего начать изучение Python: 3 первых ша

In [ ]:
URL = 'https://netology.ru/blog/?infinity=scrolling'
params = {
    'page': 2,
    'query_args[s]': 'python'
}
req = requests.get(URL, params=params)

soup = BeautifulSoup(req.json()['html'])
articles = soup.find_all('article', class_='status-publish')
netology_blog = pd.DataFrame()
for article in articles:
    title = article.find('h2', class_='entry-title').text
    link = article.find('h2', class_='entry-title').find('a').get('href')
    date = article.find('span', class_='posted-on').text.strip()
    if article.find('div', class_='entry-cats'):
        category = article.find('div', class_='entry-cats').text
    else:
        category = 'Новость'
    row = {'date': date, 'title': title, 'link': link, 'category':category}
    netology_blog = pd.concat([netology_blog, pd.DataFrame([row])]) 
netology_blog

,date,title,link,category
0,15 ноября 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/11-2022-data-science-...,Книжная полка
0,15 ноября 2022,Ручное тестирование лендинга: что нужно знать ...,https://netology.ru/blog/11-2022-manual-testin...,Продуктивность Руководство
0,28 октября 2022,Онбординг: каким он должен быть для работника ...,https://netology.ru/blog/10-2022-onboarding,Разобраться
0,23 сентября 2022,"Кто такой QA-инженер, чем он занимается и скол...",https://netology.ru/blog/09-2022-who-is-qa,Современные профессии
0,17 августа 2022,"Data Science-дайджест: что почитать, посмотрет...",https://netology.ru/blog/08-2022-data-science-...,Книжная полка
0,5 августа 2022,"Мобильный разработчик: что это за профессия, ч...",https://netology.ru/blog/08-2022-ios-android-d...,Современные профессии
